## Bitcoin data set

In [48]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime

In [49]:
data_bt_usd = pd.read_csv("csv files/BTC_USD.csv")
data_bt_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   1827 non-null   object
 1   Şimdi   1827 non-null   object
 2   Açılış  1827 non-null   object
 3   Yüksek  1827 non-null   object
 4   Düşük   1827 non-null   object
 5   Hac.    1827 non-null   object
 6   Fark %  1827 non-null   object
dtypes: object(7)
memory usage: 100.0+ KB


In [50]:
df = data_bt_usd.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)
df

,Tarih,Şimdi
0,01.01.2023,"16.593,0"
1,31.12.2022,"16.564,0"
2,30.12.2022,"16.618,0"
3,29.12.2022,"16.658,0"
4,28.12.2022,"16.558,0"
...,...,...
1822,05.01.2018,"16.917,0"
1823,04.01.2018,"15.160,0"
1824,03.01.2018,"15.155,0"
1825,02.01.2018,"14.709,8"


In [51]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '')))

## LSTM

In [52]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])




[16.593 16.564 16.618 16.658 16.558 16.719 16.946] 16.847
[16.564 16.618 16.658 16.558 16.719 16.946 16.847] 16.864
[16.618 16.658 16.558 16.719 16.946 16.847 16.864] 16.802
[16.658 16.558 16.719 16.946 16.847 16.864 16.802] 16.838
[16.558 16.719 16.946 16.847 16.864 16.802 16.838] 16.845
[16.719 16.946 16.847 16.864 16.802 16.838 16.845] 16.925
[16.946 16.847 16.864 16.802 16.838 16.845 16.925] 16.448
[16.847 16.864 16.802 16.838 16.845 16.925 16.448] 16.757
[16.864 16.802 16.838 16.845 16.925 16.448 16.757] 16.793
[16.802 16.838 16.845 16.925 16.448 16.757 16.793] 16.642
[16.838 16.845 16.925 16.448 16.757 16.793 16.642] 17.372
[16.845 16.925 16.448 16.757 16.793 16.642 17.372] 17.81
[16.925 16.448 16.757 16.793 16.642 17.372 17.81 ] 17.785
[16.448 16.757 16.793 16.642 17.372 17.81  17.785] 17.228
[16.757 16.793 16.642 17.372 17.81  17.785 17.228] 17.109
[16.793 16.642 17.372 17.81  17.785 17.228 17.109] 17.132
[16.642 17.372 17.81  17.785 17.228 17.109 17.132] 17.135
[17.372 17.81  

In [53]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
57/57 [==============================] - 1s 6ms/step - loss: 49.1590
Epoch 2/200
57/57 [==============================] - 0s 6ms/step - loss: 3.6374
Epoch 3/200
57/57 [==============================] - 0s 6ms/step - loss: 3.5085
Epoch 4/200
57/57 [==============================] - 0s 6ms/step - loss: 2.9420
Epoch 5/200
57/57 [==============================] - 0s 6ms/step - loss: 2.9442
Epoch 6/200
57/57 [==============================] - 0s 6ms/step - loss: 2.9821
Epoch 7/200
57/57 [==============================] - 0s 6ms/step - loss: 2.1762
Epoch 8/200
57/57 [==============================] - 0s 6ms/step - loss: 2.2508
Epoch 9/200
57/57 [==============================] - 0s 6ms/step - loss: 1.8797
Epoch 10/200
57/57 [==============================] - 0s 6ms/step - loss: 1.7255
Epoch 11/200
57/57 [==============================] - 0s 6ms/step - loss: 1.8311
Epoch 12/200
57/57 [==============================] - 0s 6ms/step - loss: 1.7300
Epoch 13/200
57/57 [================

In [54]:
# demonstrate prediction
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], df['Şimdi'][6]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

## Daily 

In [55]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0]*1000), "$ Bitcoin/USD in" ,x.strftime('%x'))

Prediction: 16863 $ Bitcoin/USD in 01/01/23


## Weekly

In [56]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3], df['Şimdi'][4], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2] ,float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_5), float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_6 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_7 = model.predict(x_input, verbose=0)

x_input = array([float(yhat_7), float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_8 = model.predict(x_input, verbose=0)


yhat   = float(yhat)*1000
yhat_2 = float(yhat_2)*1000
yhat_3 = float(yhat_3)*1000
yhat_4 = float(yhat_4)*1000
yhat_5 = float(yhat_5)*1000
yhat_6 = float(yhat_6)*1000
yhat_7 = float(yhat_7)*1000
yhat_8 = float(yhat_8)*1000


In [57]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ Bitcoin/USD ")


Monday: 16863.308 Tuesday : 16767.965 Wednesday: 16756.71 Thursday: 16776.169 Friday: 16753.323  $ Bitcoin/USD 


In [58]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Week -> Prediction: {round(yhat_8,2)} $ Bitcoin/USD ")


Next Week -> Prediction: 16772.13 $ Bitcoin/USD 


## Monthly -> Burada bir sorun var. Garip bir hatası bulunmakta.

In [59]:
import array
exchange = array.array('f',[])

for i in range(len(df)):
    if i % 7 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])

In [60]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[16.593 16.847 16.757 17.109 17.128 16.432 16.281] 16.344999313354492
[16.847 16.757 17.109 17.128 16.432 16.281 16.345] 20.93199920654297
[16.757 17.109 17.128 16.432 16.281 16.345 20.932] 20.64699935913086
[17.109 17.128 16.432 16.281 16.345 20.932 20.647] 19.584999084472656
[17.128 16.432 16.281 16.345 20.932 20.647 19.585] 19.268999099731445
[16.432 16.281 16.345 20.932 20.647 19.585 19.269] 19.444400787353516
[16.281  16.345  20.932  20.647  19.585  19.269  19.4444] 19.06599998474121
[16.345  20.932  20.647  19.585  19.269  19.4444 19.066 ] 18.816999435424805
[20.932  20.647  19.585  19.269  19.4444 19.066  18.817 ] 19.423999786376953
[20.647  19.585  19.269  19.4444 19.066  18.817  19.424 ] 21.826000213623047
[19.585  19.269  19.4444 19.066  18.817  19.424  21.826 ] 20.00200080871582
[19.269  19.4444 19.066  18.817  19.424  21.826  20.002 ] 19.55780029296875
[19.4444 19.066  18.817  19.424  21.826  20.002  19.5578] 21.510000228881836
[19.066  18.817  19.424  21.826  20.002  19.55

In [61]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
8/8 [==============================] - 1s 6ms/step - loss: 356.1135
Epoch 2/200
8/8 [==============================] - 0s 6ms/step - loss: 41.4973
Epoch 3/200
8/8 [==============================] - 0s 6ms/step - loss: 25.0797
Epoch 4/200
8/8 [==============================] - 0s 6ms/step - loss: 20.5071
Epoch 5/200
8/8 [==============================] - 0s 6ms/step - loss: 15.8066
Epoch 6/200
8/8 [==============================] - 0s 6ms/step - loss: 13.4616
Epoch 7/200
8/8 [==============================] - 0s 6ms/step - loss: 14.4644
Epoch 8/200
8/8 [==============================] - 0s 7ms/step - loss: 11.2138
Epoch 9/200
8/8 [==============================] - 0s 7ms/step - loss: 11.0646
Epoch 10/200
8/8 [==============================] - 0s 7ms/step - loss: 11.0159
Epoch 11/200
8/8 [==============================] - 0s 6ms/step - loss: 11.4776
Epoch 12/200
8/8 [==============================] - 0s 6ms/step - loss: 14.3080
Epoch 13/200
8/8 [==============================

In [62]:
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4],exchange[5],exchange[6]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [63]:
# onluk olarak yukarı yuvarlandı.
print("Next Month -> Prediction:",round(yhat[0][0]*1000,3), "$ Bitcoin/USD in", x.strftime('%B'))


Next Month -> Prediction: 16577.795 $ Bitcoin/USD in January


## Year


In [64]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 30 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [65]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[16.593 17.091 20.168 19.638 19.835] 22.6117000579834
[17.091  20.168  19.638  19.835  22.6117] 20.16900062561035
[20.168  19.638  19.835  22.6117 20.169 ] 29.92340087890625
[19.638  19.835  22.6117 20.169  29.9234] 36.040000915527344
[19.835  22.6117 20.169  29.9234 36.04  ] 43.19900131225586
[22.6117 20.169  29.9234 36.04   43.199 ] 38.03900146484375
[20.169  29.9234 36.04   43.199  38.039 ] 41.45399856567383
[29.9234 36.04   43.199  38.039  41.454 ] 43.111000061035156
[36.04  43.199 38.039 41.454 43.111] 50.63399887084961
[43.199 38.039 41.454 43.111 50.634] 63.25910186767578
[38.039  41.454  43.111  50.634  63.2591] 53.92499923706055
[41.454  43.111  50.634  63.2591 53.925 ] 46.0880012512207
[43.111  50.634  63.2591 53.925  46.088 ] 46.292999267578125
[50.634  63.2591 53.925  46.088  46.293 ] 33.50600051879883
[63.2591 53.925  46.088  46.293  33.506 ] 36.649200439453125
[53.925  46.088  46.293  33.506  36.6492] 56.71500015258789
[46.088  46.293  33.506  36.6492 56.715 ] 60.03480148

In [66]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
2/2 [==============================] - 1s 8ms/step - loss: 776.1872
Epoch 2/300
2/2 [==============================] - 0s 32ms/step - loss: 439.0313
Epoch 3/300
2/2 [==============================] - 0s 22ms/step - loss: 473.8316
Epoch 4/300
2/2 [==============================] - 0s 7ms/step - loss: 331.7593
Epoch 5/300
2/2 [==============================] - 0s 6ms/step - loss: 229.7444
Epoch 6/300
2/2 [==============================] - 0s 7ms/step - loss: 285.0637
Epoch 7/300
2/2 [==============================] - 0s 7ms/step - loss: 300.5489
Epoch 8/300
2/2 [==============================] - 0s 7ms/step - loss: 286.3611
Epoch 9/300
2/2 [==============================] - 0s 7ms/step - loss: 247.0713
Epoch 10/300
2/2 [==============================] - 0s 7ms/step - loss: 200.1903
Epoch 11/300
2/2 [==============================] - 0s 6ms/step - loss: 151.4112
Epoch 12/300
2/2 [==============================] - 0s 6ms/step - loss: 129.9305
Epoch 13/300
2/2 [=================

In [67]:
# onluk olarak yukarı yuvarlandı.
print(
    f"Next Year -> Prediction: {round(float(yhat[0][0]*1000),3)}  ₺ Bitcoin/USD in {x.strftime('%Y')} ")


Next Year -> Prediction: 20879.662  ₺ Bitcoin/USD in 2023 
